In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                url = 'https://ecatalog.smpcorp.com/v2/fs/api/image/getallimages?partNum=' + input_.loc[a, 'Part_Number'] + '&brand=' + input_.loc[a, 'Brand'] + '&zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960'

                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'JOIN_PNB': input_.loc[a, 'JOIN_PNB'],
                                         'Src': '' if len(json_data) == 0 else json_data[0]['image_BG_Url']}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number'],
                                     'Brand': input_.loc[a, 'Brand']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'JOIN_PNB'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(50):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['JOIN_PNB'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./src-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Part_Number', 'Brand'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./src_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：3242

[ok] - 11012;FS
[尝试次数：1] - [剩余数量：3192] - [当前时间：14:53:32]

[ok] - 15139;FS
[尝试次数：1] - [剩余数量：3191] - [当前时间：14:53:32]

[ok] - 11068;FS
[尝试次数：1] - [剩余数量：3190] - [当前时间：14:53:32]

[ok] - 14261;FS
[尝试次数：1] - [剩余数量：3189] - [当前时间：14:53:32]

[ok] - 15151;FS
[尝试次数：1] - [剩余数量：3188] - [当前时间：14:53:32]

[ok] - 14264;FS
[尝试次数：1] - [剩余数量：3187] - [当前时间：14:53:32]

[ok] - 15133;FS
[尝试次数：1] - [剩余数量：3186] - [当前时间：14:53:32]

[ok] - 14263;FS
[尝试次数：1] - [剩余数量：3185] - [当前时间：14:53:32]

[ok] - 15128;FS
[尝试次数：1] - [剩余数量：3184] - [当前时间：14:53:32]

[ok] - 15135;FS
[尝试次数：1] - [剩余数量：3183] - [当前时间：14:53:32]

[ok] - 14233;FS
[尝试次数：1] - [剩余数量：3182] - [当前时间：14:53:32]

[ok] - 14232;FS
[尝试次数：1] - [剩余数量：3181] - [当前时间：14:53:32]

[ok] - 14234;FS
[尝试次数：1] - [剩余数量：3180] - [当前时间：14:53:32]

[ok] - 15105;FS
[尝试次数：1] - [剩余数量：3179] - [当前时间：14:53:32]

[ok] - 15130;FS
[尝试次数：1] - [剩余数量：3178] - [当前时间：14:53:32]

[ok] - 15107;FS
[尝试次数：1] - [剩余数量：3177] - [当前时间：14:53:32]

[ok] - 14231;FS
[尝试次数：1] - [剩余数量：3176] - [当前时间：14:53:32]

[ok]

[ok] - 55053;FS
[尝试次数：1] - [剩余数量：3048] - [当前时间：14:53:34]

[ok] - 55058;FS
[尝试次数：1] - [剩余数量：3047] - [当前时间：14:53:34]

[ok] - 55055;FS
[尝试次数：1] - [剩余数量：3046] - [当前时间：14:53:34]

[ok] - 55006;FS
[尝试次数：1] - [剩余数量：3045] - [当前时间：14:53:34]

[ok] - 55061;FS
[尝试次数：1] - [剩余数量：3044] - [当前时间：14:53:34]

[ok] - 55060;FS
[尝试次数：1] - [剩余数量：3043] - [当前时间：14:53:34]

[ok] - 55062;FS
[尝试次数：1] - [剩余数量：3042] - [当前时间：14:53:34]

[ok] - 55073;FS
[尝试次数：1] - [剩余数量：3041] - [当前时间：14:53:34]

[ok] - 55068;FS
[尝试次数：1] - [剩余数量：3040] - [当前时间：14:53:34]

[ok] - 55063;FS
[尝试次数：1] - [剩余数量：3039] - [当前时间：14:53:34]

[ok] - 55071;FS
[尝试次数：1] - [剩余数量：3038] - [当前时间：14:53:34]

[ok] - 55067;FS
[尝试次数：1] - [剩余数量：3037] - [当前时间：14:53:34]

[ok] - 55076;FS
[尝试次数：1] - [剩余数量：3036] - [当前时间：14:53:34]

[ok] - 55070;FS
[尝试次数：1] - [剩余数量：3035] - [当前时间：14:53:34]

[ok] - 55074;FS
[尝试次数：1] - [剩余数量：3034] - [当前时间：14:53:34]

[ok] - 55084;FS
[尝试次数：1] - [剩余数量：3033] - [当前时间：14:53:34]

[ok] - 55078;FS
[尝试次数：1] - [剩余数量：3032] - [当前时间：14:53:34]

[ok] - 55079;F

[ok] - 55237;FS
[尝试次数：1] - [剩余数量：2895] - [当前时间：14:53:36]

[ok] - 55238;FS
[尝试次数：1] - [剩余数量：2894] - [当前时间：14:53:36]

[ok] - 55239;FS
[尝试次数：1] - [剩余数量：2893] - [当前时间：14:53:36]

[ok] - 55242;FS
[尝试次数：1] - [剩余数量：2892] - [当前时间：14:53:36]

[ok] - 55247;FS
[尝试次数：1] - [剩余数量：2891] - [当前时间：14:53:36]

[ok] - 55251;FS
[尝试次数：1] - [剩余数量：2890] - [当前时间：14:53:36]

[ok] - 55241;FS
[尝试次数：1] - [剩余数量：2889] - [当前时间：14:53:36]

[ok] - 55243;FS
[尝试次数：1] - [剩余数量：2888] - [当前时间：14:53:36]

[ok] - 55246;FS
[尝试次数：1] - [剩余数量：2887] - [当前时间：14:53:36]

[ok] - 55177;FS
[尝试次数：1] - [剩余数量：2886] - [当前时间：14:53:36]

[ok] - 55245;FS
[尝试次数：1] - [剩余数量：2885] - [当前时间：14:53:36]

[ok] - 55249;FS
[尝试次数：1] - [剩余数量：2884] - [当前时间：14:53:36]

[ok] - 55262;FS
[尝试次数：1] - [剩余数量：2883] - [当前时间：14:53:37]

[ok] - 55260;FS
[尝试次数：1] - [剩余数量：2882] - [当前时间：14:53:37]

[ok] - 55263;FS
[尝试次数：1] - [剩余数量：2881] - [当前时间：14:53:37]

[ok] - 55253;FS
[尝试次数：1] - [剩余数量：2880] - [当前时间：14:53:37]

[ok] - 55258;FS
[尝试次数：1] - [剩余数量：2879] - [当前时间：14:53:37]

[ok] - 55261;F

[ok] - 55433;FS
[尝试次数：1] - [剩余数量：2738] - [当前时间：14:53:39]

[ok] - 55434;FS
[尝试次数：1] - [剩余数量：2737] - [当前时间：14:53:39]

[ok] - 55435;FS
[尝试次数：1] - [剩余数量：2736] - [当前时间：14:53:39]

[ok] - 55445;FS
[尝试次数：1] - [剩余数量：2735] - [当前时间：14:53:39]

[ok] - 55437;FS
[尝试次数：1] - [剩余数量：2734] - [当前时间：14:53:39]

[ok] - 55442;FS
[尝试次数：1] - [剩余数量：2733] - [当前时间：14:53:39]

[ok] - 55436;FS
[尝试次数：1] - [剩余数量：2732] - [当前时间：14:53:39]

[ok] - 55443;FS
[尝试次数：1] - [剩余数量：2731] - [当前时间：14:53:39]

[ok] - 55451;FS
[尝试次数：1] - [剩余数量：2730] - [当前时间：14:53:39]

[ok] - 55450;FS
[尝试次数：1] - [剩余数量：2729] - [当前时间：14:53:39]

[ok] - 55438;FS
[尝试次数：1] - [剩余数量：2728] - [当前时间：14:53:39]

[ok] - 55456;FS
[尝试次数：1] - [剩余数量：2727] - [当前时间：14:53:39]

[ok] - 55461;FS
[尝试次数：1] - [剩余数量：2726] - [当前时间：14:53:40]

[ok] - 55224;FS
[尝试次数：1] - [剩余数量：2725] - [当前时间：14:53:40]

[ok] - 55439;FS
[尝试次数：1] - [剩余数量：2724] - [当前时间：14:53:40]

[ok] - 55440;FS
[尝试次数：1] - [剩余数量：2723] - [当前时间：14:53:40]

[ok] - 55441;FS
[尝试次数：1] - [剩余数量：2722] - [当前时间：14:53:40]

[ok] - 55446;F

[ok] - 55667;FS
[尝试次数：1] - [剩余数量：2595] - [当前时间：14:53:42]

[ok] - 55668;FS
[尝试次数：1] - [剩余数量：2594] - [当前时间：14:53:42]

[ok] - 55661;FS
[尝试次数：1] - [剩余数量：2593] - [当前时间：14:53:42]

[ok] - 55666;FS
[尝试次数：1] - [剩余数量：2592] - [当前时间：14:53:42]

[ok] - 55671;FS
[尝试次数：1] - [剩余数量：2591] - [当前时间：14:53:42]

[ok] - 55673;FS
[尝试次数：1] - [剩余数量：2590] - [当前时间：14:53:42]

[ok] - 55665;FS
[尝试次数：1] - [剩余数量：2589] - [当前时间：14:53:42]

[ok] - 55669;FS
[尝试次数：1] - [剩余数量：2588] - [当前时间：14:53:42]

[ok] - 55672;FS
[尝试次数：1] - [剩余数量：2587] - [当前时间：14:53:42]

[ok] - 55664;FS
[尝试次数：1] - [剩余数量：2586] - [当前时间：14:53:42]

[ok] - 55670;FS
[尝试次数：1] - [剩余数量：2585] - [当前时间：14:53:42]

[ok] - 55663;FS
[尝试次数：1] - [剩余数量：2584] - [当前时间：14:53:42]

[ok] - 55677;FS
[尝试次数：1] - [剩余数量：2583] - [当前时间：14:53:42]

[ok] - 55680;FS
[尝试次数：1] - [剩余数量：2582] - [当前时间：14:53:42]

[ok] - 55691;FS
[尝试次数：1] - [剩余数量：2581] - [当前时间：14:53:42]

[ok] - 55676;FS
[尝试次数：1] - [剩余数量：2580] - [当前时间：14:53:42]

[ok] - 55662;FS
[尝试次数：1] - [剩余数量：2579] - [当前时间：14:53:42]

[ok] - 55686;F

...

[ok] - 66271;FS
[尝试次数：1] - [剩余数量：646] - [当前时间：14:54:11]

[ok] - 66288;FS
[尝试次数：1] - [剩余数量：645] - [当前时间：14:54:11]

[ok] - 66350;FS
[尝试次数：1] - [剩余数量：644] - [当前时间：14:54:11]

[ok] - 66358;FS
[尝试次数：1] - [剩余数量：643] - [当前时间：14:54:11]

[ok] - 66339;FS
[尝试次数：1] - [剩余数量：642] - [当前时间：14:54:11]

[ok] - 66347;FS
[尝试次数：1] - [剩余数量：641] - [当前时间：14:54:11]

[ok] - 66341;FS
[尝试次数：1] - [剩余数量：640] - [当前时间：14:54:11]

[ok] - 66343;FS
[尝试次数：1] - [剩余数量：639] - [当前时间：14:54:11]

[ok] - 66342;FS
[尝试次数：1] - [剩余数量：638] - [当前时间：14:54:11]

[ok] - 66345;FS
[尝试次数：1] - [剩余数量：637] - [当前时间：14:54:11]

[ok] - 66346;FS
[尝试次数：1] - [剩余数量：636] - [当前时间：14:54:11]

[ok] - 66353;FS
[尝试次数：1] - [剩余数量：635] - [当前时间：14:54:11]

[ok] - 66348;FS
[尝试次数：1] - [剩余数量：634] - [当前时间：14:54:11]

[ok] - 66355;FS
[尝试次数：1] - [剩余数量：633] - [当前时间：14:54:11]

[ok] - 66298;FS
[尝试次数：1] - [剩余数量：632] - [当前时间：14:54:11]

[ok] - 66354;FS
[尝试次数：1] - [剩余数量：631] - [当前时间：14:54:11]

[ok] - 66361;FS
[尝试次数：1] - [剩余数量：630] - [当前时间：14:54:11]

[ok] - 66360;FS
[尝试次数：1] - [剩余数

[ok] - 66499;FS
[尝试次数：1] - [剩余数量：490] - [当前时间：14:54:14]

[ok] - 66471;FS
[尝试次数：1] - [剩余数量：489] - [当前时间：14:54:14]

[ok] - 66500;FS
[尝试次数：1] - [剩余数量：488] - [当前时间：14:54:14]

[ok] - 66488;FS
[尝试次数：1] - [剩余数量：487] - [当前时间：14:54:14]

[ok] - 66501;FS
[尝试次数：1] - [剩余数量：486] - [当前时间：14:54:14]

[ok] - 66485;FS
[尝试次数：1] - [剩余数量：485] - [当前时间：14:54:14]

[ok] - 66504;FS
[尝试次数：1] - [剩余数量：484] - [当前时间：14:54:14]

[ok] - 66453;FS
[尝试次数：1] - [剩余数量：483] - [当前时间：14:54:14]

[ok] - 66503;FS
[尝试次数：1] - [剩余数量：482] - [当前时间：14:54:14]

[ok] - 66508;FS
[尝试次数：1] - [剩余数量：481] - [当前时间：14:54:14]

[ok] - 66505;FS
[尝试次数：1] - [剩余数量：480] - [当前时间：14:54:14]

[ok] - 66502;FS
[尝试次数：1] - [剩余数量：479] - [当前时间：14:54:14]

[ok] - 66506;FS
[尝试次数：1] - [剩余数量：478] - [当前时间：14:54:14]

[ok] - 66510;FS
[尝试次数：1] - [剩余数量：477] - [当前时间：14:54:14]

[ok] - 66509;FS
[尝试次数：1] - [剩余数量：476] - [当前时间：14:54:14]

[ok] - 66511;FS
[尝试次数：1] - [剩余数量：475] - [当前时间：14:54:14]

[ok] - 66512;FS
[尝试次数：1] - [剩余数量：474] - [当前时间：14:54:14]

[ok] - 66515;FS
[尝试次数：1] - [剩余数

[ok] - 66648;FS
[尝试次数：1] - [剩余数量：345] - [当前时间：14:54:16]

[ok] - 66646;FS
[尝试次数：1] - [剩余数量：344] - [当前时间：14:54:16]

[ok] - 66651;FS
[尝试次数：1] - [剩余数量：343] - [当前时间：14:54:16]

[ok] - 66650;FS
[尝试次数：1] - [剩余数量：342] - [当前时间：14:54:16]

[ok] - 66657;FS
[尝试次数：1] - [剩余数量：341] - [当前时间：14:54:16]

[ok] - 66654;FS
[尝试次数：1] - [剩余数量：340] - [当前时间：14:54:16]

[ok] - 66658;FS
[尝试次数：1] - [剩余数量：339] - [当前时间：14:54:16]

[ok] - 66652;FS
[尝试次数：1] - [剩余数量：338] - [当前时间：14:54:16]

[ok] - 66653;FS
[尝试次数：1] - [剩余数量：337] - [当前时间：14:54:16]

[ok] - 66655;FS
[尝试次数：1] - [剩余数量：336] - [当前时间：14:54:16]

[ok] - 66656;FS
[尝试次数：1] - [剩余数量：335] - [当前时间：14:54:16]

[ok] - 66662;FS
[尝试次数：1] - [剩余数量：334] - [当前时间：14:54:16]

[ok] - 66661;FS
[尝试次数：1] - [剩余数量：333] - [当前时间：14:54:16]

[ok] - 66663;FS
[尝试次数：1] - [剩余数量：332] - [当前时间：14:54:16]

[ok] - 66664;FS
[尝试次数：1] - [剩余数量：331] - [当前时间：14:54:16]

[ok] - 66665;FS
[尝试次数：1] - [剩余数量：330] - [当前时间：14:54:16]

[ok] - 66607;FS
[尝试次数：1] - [剩余数量：329] - [当前时间：14:54:16]

[ok] - 66631;FS
[尝试次数：1] - [剩余数

[ok] - 66797;FS
[尝试次数：1] - [剩余数量：191] - [当前时间：14:54:18]

[ok] - 66805;FS
[尝试次数：1] - [剩余数量：190] - [当前时间：14:54:18]

[ok] - 66808;FS
[尝试次数：1] - [剩余数量：189] - [当前时间：14:54:18]

[ok] - 66807;FS
[尝试次数：1] - [剩余数量：188] - [当前时间：14:54:18]

[ok] - 66809;FS
[尝试次数：1] - [剩余数量：187] - [当前时间：14:54:18]

[ok] - 66812;FS
[尝试次数：1] - [剩余数量：186] - [当前时间：14:54:18]

[ok] - 66811;FS
[尝试次数：1] - [剩余数量：185] - [当前时间：14:54:18]

[ok] - 66815;FS
[尝试次数：1] - [剩余数量：184] - [当前时间：14:54:18]

[ok] - 66814;FS
[尝试次数：1] - [剩余数量：183] - [当前时间：14:54:18]

[ok] - 66821;FS
[尝试次数：1] - [剩余数量：182] - [当前时间：14:54:18]

[ok] - 66806;FS
[尝试次数：1] - [剩余数量：181] - [当前时间：14:54:18]

[ok] - 66810;FS
[尝试次数：1] - [剩余数量：180] - [当前时间：14:54:19]

[ok] - 66819;FS
[尝试次数：1] - [剩余数量：179] - [当前时间：14:54:19]

[ok] - 66813;FS
[尝试次数：1] - [剩余数量：178] - [当前时间：14:54:19]

[ok] - 66822;FS
[尝试次数：1] - [剩余数量：177] - [当前时间：14:54:19]

[ok] - 66823;FS
[尝试次数：1] - [剩余数量：176] - [当前时间：14:54:19]

[ok] - 66826;FS
[尝试次数：1] - [剩余数量：175] - [当前时间：14:54:19]

[ok] - 66820;FS
[尝试次数：1] - [剩余数

[ok] - 66957;FS
[尝试次数：1] - [剩余数量：46] - [当前时间：14:54:20]

[ok] - 66967;FS
[尝试次数：1] - [剩余数量：45] - [当前时间：14:54:21]

[ok] - 66962;FS
[尝试次数：1] - [剩余数量：44] - [当前时间：14:54:21]

[ok] - 66958;FS
[尝试次数：1] - [剩余数量：43] - [当前时间：14:54:21]

[ok] - 66959;FS
[尝试次数：1] - [剩余数量：42] - [当前时间：14:54:21]

[ok] - 66963;FS
[尝试次数：1] - [剩余数量：41] - [当前时间：14:54:21]

[ok] - 66966;FS
[尝试次数：1] - [剩余数量：40] - [当前时间：14:54:21]

[ok] - 66960;FS
[尝试次数：1] - [剩余数量：39] - [当前时间：14:54:21]

[ok] - 66964;FS
[尝试次数：1] - [剩余数量：38] - [当前时间：14:54:21]

[ok] - 66961;FS
[尝试次数：1] - [剩余数量：37] - [当前时间：14:54:21]

[ok] - 66969;FS
[尝试次数：1] - [剩余数量：36] - [当前时间：14:54:21]

[ok] - 66970;FS
[尝试次数：1] - [剩余数量：35] - [当前时间：14:54:21]

[ok] - 66974;FS
[尝试次数：1] - [剩余数量：34] - [当前时间：14:54:21]

[ok] - 66972;FS
[尝试次数：1] - [剩余数量：33] - [当前时间：14:54:21]

[ok] - 66971;FS
[尝试次数：1] - [剩余数量：32] - [当前时间：14:54:21]

[ok] - 66968;FS
[尝试次数：1] - [剩余数量：31] - [当前时间：14:54:21]

[ok] - 66973;FS
[尝试次数：1] - [剩余数量：30] - [当前时间：14:54:21]

[ok] - 66984;FS
[尝试次数：1] - [剩余数量：29] - [当前时间：14: